In [163]:
#Though it works as expected, its loss makes it unviable for forecasting
#usage, trying to optimize...
#TODO 

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.regularizers import l2
import pandas as pd
import numpy as np
from predict import date_to_int
from sklearn.preprocessing import MinMaxScaler

TRAIN_RATIO = 0.8
TIMESTEP = 12


In [167]:
def create_model(input_shape):
    
    model = Sequential()
    model.add(LSTM(units=50, input_shape=input_shape, return_sequences=False))
    model.add(Dense(units=1))
    model.compile(optimizer=Adam(learning_rate=0.001),loss='mean_squared_error')
        
    return model

In [1]:
df = pd.read_csv('data/yf_monthly/AAPL.csv')
df['Date'] = [date_to_int(date) for date in df['Date']]

X_sc = MinMaxScaler()
scaled_X = X_sc.fit_transform(df)

y_sc = MinMaxScaler()
scaled_y = y_sc.fit_transform(df[['Close']])

NameError: name 'pd' is not defined

In [169]:
def create_sequences(scaled_X, scaled_y, timestep):
    X, y = [], []
    for i in range(len(scaled_X) - timestep):
        X.append(scaled_X[i:i + timestep]) #Using all data between timesteps
        y.append(scaled_y[i + timestep]) #Using next timesteps close data as target
    
    return np.array(X), np.array(y)

In [170]:

X, y = create_sequences(scaled_X, scaled_y, TIMESTEP)

split_point = int(TRAIN_RATIO * len(X))
X_train, y_train = X[:split_point], y[:split_point]
X_test, y_test = X[split_point:], y[split_point:]



In [171]:
model = create_model((X_train.shape[1], X_train.shape[2]))
model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
12/12 [==============================] - 2s 8ms/step - loss: 7.3127e-04
Epoch 2/20
12/12 [==============================] - 0s 7ms/step - loss: 2.2222e-04
Epoch 3/20
12/12 [==============================] - 0s 8ms/step - loss: 1.0938e-04
Epoch 4/20
12/12 [==============================] - 0s 7ms/step - loss: 7.8815e-05
Epoch 5/20
12/12 [==============================] - 0s 7ms/step - loss: 6.8249e-05
Epoch 6/20
12/12 [==============================] - 0s 8ms/step - loss: 6.5894e-05
Epoch 7/20
12/12 [==============================] - 0s 7ms/step - loss: 5.5930e-05
Epoch 8/20
12/12 [==============================] - 0s 7ms/step - loss: 6.9053e-05
Epoch 9/20
12/12 [==============================] - 0s 7ms/step - loss: 8.0188e-05
Epoch 10/20
12/12 [==============================] - 0s 7ms/step - loss: 5.8745e-05
Epoch 11/20
12/12 [==============================] - 0s 8ms/step - loss: 5.8811e-05
Epoch 12/20
12/12 [==============================] - 0s 7ms/step - loss: 5.8679e-05
E

In [172]:
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

3/3 [==============================] - 0s 5ms/step - loss: 0.0073
Test Loss: 0.007331139408051968


In [176]:
#Testing purposes
last_sequence = X_test[-1].reshape(1, X_test[-1].shape[0], X_test[-1].shape[1])

scaled_prediction = model.predict(last_sequence)

prediction = y_sc.inverse_transform(scaled_prediction)

last_y = y_sc.inverse_transform([y_test[-1]])
print(last_y, prediction)




1/1 [==============================] - 0s 39ms/step
[[180.75]] [[163.30807]]
